In [2]:
!pip install opencv-python opencv-python-headless face_recognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=885babb258c6e3a4dcf51ab5e712ddcfaedfa1efafe6e96bdffa421d43f7ba1d
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [3]:
import cv2
import face_recognition

# Load the pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the target image (the face you want to detect)
target_image = cv2.imread('/content/drive/MyDrive/dad.png')
gray_target = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

# Detect the face in the target image
target_faces = face_cascade.detectMultiScale(gray_target, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Get the target face encoding if a face is found
if len(target_faces) > 0:
    x, y, w, h = target_faces[0]  # Assuming only one face in the target image
    target_face_encoding = face_recognition.face_encodings(target_image, [(y, x + w, y + h, x)])[0]
else:
    print("No face detected in the target image.")
    target_face_encoding = None

# Proceed only if a target face encoding is available
if target_face_encoding is not None:
    # Open the video file
    video_path = '/content/drive/MyDrive/merge_video_oldman_girlwithdad.mp4'
    cap = cv2.VideoCapture(video_path)

    # Get video properties for output video
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_oldman_girlwithdad.mp4', fourcc, fps, (width, height))

    frame_counter = 0  # Keep track of frame index
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the current frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Check if any detected faces match the target face
        for (x, y, w, h) in faces:
            face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]
            matches = face_recognition.compare_faces([target_face_encoding], face_encoding, tolerance=0.4)
            if True in matches:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

                # Save the frames where the target face is found
                out.write(frame)

        frame_counter += 1

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processed and saved as 'output_oldman_girlwithdad.mp4'")
else:
    print("Target face not found, video processing skipped.")


Video processed and saved as 'output_oldman_girlwithdad.mp4'


In [ ]:
import cv2
import face_recognition
import streamlit as st
import tempfile
import os
import numpy as np
from io import BytesIO

# Function to process the video
def process_video(uploaded_video, uploaded_image):
    # Load the target image (the face you want to detect)
    target_image = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
    gray_target = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

    # Use Dlib's face detector for faster detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    target_faces = face_cascade.detectMultiScale(gray_target, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Get the target face encoding if a face is found
    if len(target_faces) > 0:
        x, y, w, h = target_faces[0]  # Assuming only one face in the target image
        target_face_encoding = face_recognition.face_encodings(target_image, [(y, x + w, y + h, x)])[0]
    else:
        st.error("No face detected in the target image.")
        return None

    # Open the video file
    video_bytes = uploaded_video.read()
    video_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    with open(video_path.name, 'wb') as f:
        f.write(video_bytes)

    cap = cv2.VideoCapture(video_path.name)

    # Get video properties for output video
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a temporary output video file
    output_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path.name, fourcc, fps, (width, height))

    # Process every 5th frame to speed up
    frame_counter = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_counter % 5 == 0:  # Process every 5th frame to speed up
            # Convert the frame to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the current frame
            faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Check if any detected faces match the target face
            for (x, y, w, h) in faces:
                face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]
                matches = face_recognition.compare_faces([target_face_encoding], face_encoding, tolerance=0.4)
                if True in matches:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    # Save the frames where the target face is found
                    out.write(frame)

        frame_counter += 1

    # Release resources
    cap.release()
    out.release()

    # Return the processed video for download
    with open(output_path.name, "rb") as f:
        video_bytes = f.read()

    # Clean up temporary files
    os.remove(video_path.name)
    os.remove(output_path.name)

    return video_bytes

# Streamlit UI
st.title("Face Detection in Videos")
st.write("Upload a video and an image (face) to detect the face and get the output video.")

# Upload components
uploaded_video = st.file_uploader("Upload a video", type=["mp4", "mov", "avi"])
uploaded_image = st.file_uploader("Upload an image (face)", type=["jpg", "png", "jpeg"])

# Button to start processing
if st.button("Process Video"):
    if uploaded_video and uploaded_image:
        # Process the video and image
        output_video = process_video(uploaded_video, uploaded_image)

        if output_video:
            # Provide download link for the processed video
            st.success("Video processed successfully!")
            st.download_button("Download Processed Video", data=output_video, file_name="output_video.mp4", mime="video/mp4")
    else:
        st.error("Please upload both a video and an image.")
